In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import set_style
import kagglehub
set_style("whitegrid")

In [2]:
path = kagglehub.dataset_download("ehallmar/daily-historical-stock-prices-1970-2018")

print("Path to dataset files:", path)

Path to dataset files: /home/hava/.cache/kagglehub/datasets/ehallmar/daily-historical-stock-prices-1970-2018/versions/1


In [3]:
prices_file = path + '/historical_stock_prices.csv'
labels_file = path + '/historical_stocks.csv'
df_prices = pd.read_csv(prices_file)
df_labels = pd.read_csv(labels_file)
df_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


In [4]:
sector = df_labels[df_labels['sector']=='TECHNOLOGY'][['ticker','name']]
sector['ticker']=sector['ticker'].astype(str)
print(sector['ticker'])
df_sector = df_prices[df_prices['ticker'].isin(sector['ticker'])].reset_index()
df_sector

6       VNET
7       TWOU
8       JOBS
22      ACIA
37      ACIW
        ... 
6434     XRX
6442    YEXT
6443     YGE
6449     ZEN
6458     ZUO
Name: ticker, Length: 607, dtype: object


,index,ticker,open,close,adj_close,low,high,volume,date
0,739,VIAV,1.137656,1.173208,1.173208,1.137656,1.244312,4385800,1993-11-17
1,752,VIAV,1.173208,1.173208,1.173208,1.173208,1.244312,2372500,1993-11-18
2,768,VIAV,1.173208,1.173208,1.173208,1.173208,1.244312,806500,1993-11-19
3,784,VIAV,1.173208,1.173208,1.173208,1.137656,1.208760,602600,1993-11-22
4,800,VIAV,1.137656,1.208760,1.208760,1.137656,1.208760,149700,1993-11-23
...,...,...,...,...,...,...,...,...,...
2254328,20954376,RMBS,11.680000,11.640000,11.640000,11.540000,11.690000,582700,2018-08-20
2254329,20954377,RMBS,11.680000,11.820000,11.820000,11.680000,11.900000,446400,2018-08-21
2254330,20954378,RMBS,11.780000,11.810000,11.810000,11.710000,11.820000,371000,2018-08-22
2254331,20954379,RMBS,11.820000,11.730000,11.730000,11.700000,11.840000,394600,2018-08-23


In [5]:
df_sector['date'] = pd.to_datetime(df_sector['date'])
df_sector['ticker'] = (df_sector['ticker']).astype(str)
dst_dates_df = pd.read_csv('DaylightSavingsTimeChangeDates_1971-2024.csv')
dst_dates_spring = pd.to_datetime(dst_dates_df.Date+" "+dst_dates_df.Year.astype(str))
dst_dates_fall = pd.to_datetime(dst_dates_df['Date.1']+" "+dst_dates_df.Year.astype(str))
dst_dates = pd.concat([dst_dates_fall,dst_dates_spring])
dst_mon = dst_dates+pd.DateOffset(1)
dst_fri = dst_dates+pd.DateOffset(-2)
dst_fall_mon = dst_dates_fall+pd.DateOffset(1)
dst_fall_fri = dst_dates_fall+pd.DateOffset(-2)
dst_spring_mon = dst_dates_spring+pd.DateOffset(1)
dst_spring_fri = dst_dates_spring+pd.DateOffset(-2)

In [6]:
sector_dst_mon_fall = df_sector[df_sector.date.isin(dst_fall_mon)]
sector_dst_mon_fall = sector_dst_mon_fall.assign(Season='Fall')
sector_dst_mon_spring = df_sector[df_sector.date.isin(dst_spring_mon)]
sector_dst_mon_spring = sector_dst_mon_spring.assign(Season='Spring')
sector_dst_mon = pd.concat([sector_dst_mon_fall,sector_dst_mon_spring])
sector_dst_fri = df_sector[df_sector.date.isin(dst_fri)]

for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_dst_mon[sector_dst_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_dst_fri[sector_dst_fri.ticker.astype(str)==tkr]
    sector_dst_mon = sector_dst_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_dst_fri = sector_dst_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
    #finance_dst_mon = finance_dst_mon.drop(finance_dst_mon[(finance_dst_mon.ticker.astype(str)==tkr  finance_dst_mon['date'].isin(finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr].date+pd.DateOffset(3))==False)].index)
    #finance_dst_fri = finance_dst_fri.drop(finance_dst_fri[(finance_dst_fri['ticker'==tkr] & finance_dst_fri['date'].isin(finance_dst_mon['ticker'==tkr].date+pd.DateOffset(-3))==False)].index)
print(len(sector_dst_mon))
print(len(sector_dst_fri))

16922
16922


In [7]:
sector_dst_fri = sector_dst_fri.reset_index()
sector_dst_mon = sector_dst_mon.reset_index()

In [8]:
sector_dst = pd.DataFrame()
sector_dst['ticker'] = sector_dst_mon['ticker']
sector_dst['mon_date'] = sector_dst_mon['date']
sector_dst['mon_open'] = sector_dst_mon['open']
sector_dst['mon_close'] = sector_dst_mon['close']
sector_dst['mon_adj_close'] = sector_dst_mon['adj_close']
sector_dst['mon_low'] = sector_dst_mon['low']
sector_dst['mon_high'] = sector_dst_mon['high']
sector_dst['mon_volume'] = sector_dst_mon['volume']
sector_dst['fri_date'] = sector_dst_fri['date']
sector_dst['fri_open'] = sector_dst_fri['open']
sector_dst['fri_close'] = sector_dst_fri['close']
sector_dst['fri_adj_close'] = sector_dst_fri['adj_close']
sector_dst['fri_low'] = sector_dst_fri['low']
sector_dst['fri_high'] = sector_dst_fri['high']
sector_dst['fri_volume'] = sector_dst_fri['volume']
sector_dst['Season'] = sector_dst_mon['Season']
sector_dst

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume,Season
0,AMSWA,1983-10-31,4.493827,4.444445,1.685465,4.444445,4.592593,21100,1983-04-22,4.592593,4.493827,1.704193,4.493827,4.641975,161200,Fall
1,AAPL,1981-10-26,0.339286,0.339286,0.015323,0.339286,0.341518,6820800,1981-04-24,0.522321,0.517857,0.023388,0.517857,0.522321,8764000,Fall
2,AMSWA,1984-10-29,2.839506,2.839506,1.076826,2.839506,2.839506,81900,1983-10-28,4.740741,4.493827,1.704193,4.493827,4.740741,72100,Fall
3,VIAV,1994-10-31,1.919795,1.884243,1.884243,1.813140,1.919795,567400,1984-04-27,2.765432,2.716049,1.030007,2.567901,2.765432,23500,Fall
4,AAPL,1982-11-01,0.453125,0.477679,0.021573,0.448661,0.482143,26090400,1981-10-23,0.341518,0.339286,0.015323,0.339286,0.341518,6977600,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16917,RMBS,2014-03-10,9.660000,9.870000,9.870000,9.580000,9.940000,1032300,2016-03-11,13.250000,13.250000,13.250000,13.150000,13.350000,488900,Spring
16918,RMBS,2015-03-09,12.280000,12.390000,12.390000,12.190000,12.450000,445100,2016-11-04,11.900000,11.930000,11.930000,11.810000,12.060000,597000,Spring
16919,RMBS,2016-03-14,13.170000,12.930000,12.930000,12.810000,13.310000,942500,2017-03-10,13.160000,13.130000,13.130000,13.070000,13.250000,369500,Spring
16920,RMBS,2017-03-13,13.100000,13.100000,13.100000,13.000000,13.170000,475400,2017-11-03,14.690000,14.530000,14.530000,14.460000,14.690000,580900,Spring


In [9]:
sector_mon = df_sector[df_sector.date.dt.day_name()=='Monday']
sector_mon = sector_mon.drop(sector_mon[sector_mon.date.isin(dst_mon)].index)
sector_fri = df_sector[df_sector.date.dt.day_name()=='Friday']
sector_fri = sector_fri.drop(sector_fri[sector_fri.date.isin(dst_fri)].index)
for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_mon[sector_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_fri[sector_fri.ticker.astype(str)==tkr]
    sector_mon = sector_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_fri = sector_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
print(len(sector_mon))
print(len(sector_fri))

385220
385220


In [10]:
sector_mon = sector_mon.reset_index()
sector_fri = sector_fri.reset_index()

In [11]:
sector_wkd = pd.DataFrame()
sector_wkd['ticker'] = sector_mon['ticker']
sector_wkd['mon_date'] = sector_mon['date']
sector_wkd['mon_open'] = sector_mon['open']
sector_wkd['mon_close'] = sector_mon['close']
sector_wkd['mon_adj_close'] = sector_mon['adj_close']
sector_wkd['mon_low'] = sector_mon['low']
sector_wkd['mon_high'] = sector_mon['high']
sector_wkd['mon_volume'] = sector_mon['volume']
sector_wkd['fri_date'] = sector_fri['date']
sector_wkd['fri_open'] = sector_fri['open']
sector_wkd['fri_close'] = sector_fri['close']
sector_wkd['fri_adj_close'] = sector_fri['adj_close']
sector_wkd['fri_low'] = sector_fri['low']
sector_wkd['fri_high'] = sector_fri['high']
sector_wkd['fri_volume'] = sector_fri['volume']
sector_wkd

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume
0,VIAV,1993-11-22,1.173208,1.173208,1.173208,1.137656,1.208760,602600,1993-11-19,1.173208,1.173208,1.173208,1.173208,1.244312,806500
1,VIAV,1993-11-29,1.137656,1.208760,1.208760,1.137656,1.208760,63200,1993-11-26,1.208760,1.208760,1.208760,1.208760,1.208760,700
2,VIAV,1993-12-06,1.173208,1.244312,1.244312,1.173208,1.244312,80100,1993-12-03,1.244312,1.208760,1.208760,1.208760,1.244312,152500
3,AAPL,1980-12-15,0.488839,0.486607,0.021977,0.486607,0.488839,43971200,1980-12-12,0.513393,0.513393,0.023186,0.513393,0.515625,117258400
4,AMSWA,1983-03-07,4.543210,4.493827,1.704193,4.493827,4.592593,118300,1983-03-04,4.543210,4.543210,1.722921,4.543210,4.592593,532500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385215,RMBS,2018-07-23,13.290000,13.140000,13.140000,13.050000,13.430000,908100,2018-07-20,13.180000,13.300000,13.300000,13.150000,13.420000,736500
385216,RMBS,2018-07-30,13.490000,13.120000,13.120000,13.120000,13.500000,944800,2018-07-27,13.450000,13.390000,13.390000,13.380000,13.610000,727400
385217,RMBS,2018-08-06,12.550000,12.600000,12.600000,12.490000,12.660000,1026100,2018-08-03,12.390000,12.530000,12.530000,12.390000,12.650000,862800
385218,RMBS,2018-08-13,12.590000,12.360000,12.360000,12.340000,12.610000,555400,2018-08-10,12.620000,12.580000,12.580000,12.530000,12.690000,800700


In [12]:
sector_dst.to_csv('DST_Tech_Stocks.csv')
sector_wkd.to_csv('Weekends_Tech_Stocks.csv')